# Songs Lyrics Sentimental Analysis

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
attr_lyrics_dataset_path = r'../archive/songs_with_attributes_and_lyrics.csv'
# timestep_lyrics_dataset_path = r'archive/songs_with_lyrics_and_timestamps.csv'

dataset1 = pd.read_csv(attr_lyrics_dataset_path)
# dataset2 = pd.read_csv(timestep_lyrics_dataset_path)

In [ ]:
dataset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955320 entries, 0 to 955319
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                955320 non-null  object 
 1   name              955309 non-null  object 
 2   album_name        385557 non-null  object 
 3   artists           955318 non-null  object 
 4   danceability      955320 non-null  float64
 5   energy            955320 non-null  float64
 6   key               955320 non-null  object 
 7   loudness          955320 non-null  float64
 8   mode              955320 non-null  object 
 9   speechiness       955320 non-null  float64
 10  acousticness      955320 non-null  float64
 11  instrumentalness  955320 non-null  float64
 12  liveness          955320 non-null  float64
 13  valence           955320 non-null  float64
 14  tempo             955320 non-null  float64
 15  duration_ms       955320 non-null  float64
 16  lyrics            95

In [ ]:
dataset1.head(10)

,id,name,album_name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,lyrics
0,0Prct5TDjAnEgIqbxcldY9,!,UNDEN!ABLE,['HELLYEAH'],0.415,0.6050,7,-11.157,1,0.0575,0.001160,0.838000,0.4710,0.193,100.059,79500.0,"He said he came from Jamaica,\n he owned a cou..."
1,2ASl4wirkeYm3OWZxXKYuq,!!,NaN,Yxngxr1,0.788,0.6480,7,-9.135,0,0.3150,0.900000,0.000000,0.1760,0.287,79.998,114000.0,"Fucked a bitch, now she running with my kids\n..."
2,69lcggVPmOr9cvPx9kLiiN,!!! - Interlude,Where I Belong EP,['Glowie'],0.000,0.0354,7,-20.151,0,0.0000,0.908000,0.000000,0.4790,0.000,0.000,11413.0,"Oh, my God, I'm going crazy\n"
3,4U7dlZjg1s9pjdppqZy0fm,!!De Repente!!,Un Palo Al Agua (20 Grandes Canciones),['Rosendo'],0.657,0.8820,5,-6.340,1,0.0385,0.007400,0.000013,0.0474,0.939,123.588,198173.0,Continuamente se extraña la gente si no puede ...
4,4v1IBp3Y3rpkWmWzIlkYju,!!De Repente!!,Fuera De Lugar,['Rosendo'],0.659,0.8930,5,-8.531,1,0.0411,0.092200,0.000019,0.0534,0.951,123.600,199827.0,Continuamente se extraña la gente si no puede ...
5,5tA3ImW310llKo8EMBj2Ga,!!Noble Stabbings!!,NaN,Dillinger Four,0.171,0.9570,2,-5.749,1,0.1490,0.000029,0.000032,0.3300,0.349,175.317,197400.0,You like to stand on the other side\n Point an...
6,0fROT4kK5oTm8xO8PX6EJF,!I'll Be Back!,!I'll Be Back!,Rilès,0.823,0.6120,1,-7.767,1,0.2480,0.168000,0.000000,0.1090,0.688,142.959,178533.0,"It's been a while, shit I missed the rehab, ps..."
7,1xBFhv5faebv3mmwxx7DnS,!Lost!,NaN,Rilès,0.729,0.5520,7,-8.562,0,0.0650,0.183000,0.000000,0.1310,0.380,86.103,186197.0,I would like to give you all my time\n I would...
8,0gNNToCW3qjabgTyBSjt3H,!Que Vida! - Mono Version,NaN,['Love'],0.600,0.5400,9,-11.803,1,0.0328,0.525000,0.003050,0.1000,0.547,125.898,220560.0,With pictures and words\n Is this communicatin...
9,6LrefSCg9Vnn1ckCeJkmbA,!Viva el Mal Viva el Capital!,NaN,Elektroduendes,0.530,0.9150,0,-7.502,1,0.0370,0.003180,0.000007,0.5930,0.819,120.179,121170.0,"Seré corta y clarividente, como el cerebro del..."


In [ ]:
dataset1.isna().sum()

id                       0
name                    11
album_name          569763
artists                  2
danceability             0
energy                   0
key                      0
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness         0
liveness                 0
valence                  0
tempo                    0
duration_ms              0
lyrics                  13
dtype: int64

In [ ]:
dataset1 = dataset1.dropna(subset=['name','lyrics'])

In [ ]:
dataset1.isna().sum()

id                       0
name                     0
album_name          569744
artists                  0
danceability             0
energy                   0
key                      0
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness         0
liveness                 0
valence                  0
tempo                    0
duration_ms              0
lyrics                   0
dtype: int64

In [ ]:
dataset1['artists'].value_counts()

artists
['Various Artists']                                                                                                                                                                                                                                                                 4476
Grateful Dead                                                                                                                                                                                                                                                                       1308
['Aretha Franklin']                                                                                                                                                                                                                                                                 1067
['Bob Dylan']                                                                                                                                        

In [ ]:
dataset1['artists'].value_counts()

artists
['Various Artists']                                                                                                                                                                                                                                                                 4476
Grateful Dead                                                                                                                                                                                                                                                                       1308
['Aretha Franklin']                                                                                                                                                                                                                                                                 1067
['Bob Dylan']                                                                                                                                        

In [ ]:
dataset1['lyrics']

0         He said he came from Jamaica,\n he owned a cou...
1         Fucked a bitch, now she running with my kids\n...
2                            Oh, my God, I'm going crazy\n 
3         Continuamente se extraña la gente si no puede ...
4         Continuamente se extraña la gente si no puede ...
                                ...                        
955306    Mickey Remedy\n In a cloudless harmony\n Stay ...
955307    Looking for it over land and sea\n My heart ra...
955308    Raindrop races on the train window\n Wondering...
955309    I thought I was free\n But then the tears bega...
955310    When days are short and nights are long\n I fe...
Name: lyrics, Length: 955296, dtype: object